In [1]:
from tqdm import tqdm
from glob import glob
import os

import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import random

from torchvision.models import *
from torch.utils.data import DataLoader
from torchvision.transforms import AutoAugment
from torchvision.transforms import AutoAugmentPolicy
from torchvision.transforms import transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np

import albumentations as A
from albumentations.imgaug.transforms import IAAPiecewiseAffine
from albumentations.pytorch import ToTensorV2

from CustomLoader import CustomLoader
import timm
from utils.CosineAnnealingWarmUpRestarts import  CosineAnnealingWarmUpRestarts

from utils.utils import *
from utils.AugMix import *
from utils.CutMix import *

C:\Users\GangsuKim\Anaconda3\envs\HLAI2023\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [2]:
training_set, test_set = getDataSet('./dataset/')
set_random_seed(1813)
device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Stratic Split

In [3]:
classes = {}

for ts in training_set:
    class_n = ts.split('/')[3]

    if class_n in classes.keys():
        classes[class_n].append(ts)
    else:
        classes[class_n] = [ts]

In [5]:
dist = getDistributionDataSet(training_set)
dist = dict(sorted(dist.items(), key=lambda x:int(x[0])))

In [6]:
dist

{'0': 500,
 '1': 500,
 '2': 500,
 '3': 450,
 '4': 450,
 '5': 450,
 '6': 400,
 '7': 400,
 '8': 400,
 '9': 350,
 '10': 350,
 '11': 350,
 '12': 300,
 '13': 300,
 '14': 300,
 '15': 250,
 '16': 250,
 '17': 250,
 '18': 200,
 '19': 200,
 '20': 200,
 '21': 150,
 '22': 150,
 '23': 150,
 '24': 100,
 '25': 100,
 '26': 100,
 '27': 50,
 '28': 50,
 '29': 50}

In [7]:
fold_k = 5

fold_dict = {}

for i in range(fold_k):
    fold_dict[i] = []
    
for k in classes.keys():
    ls = classes[k]
    random.shuffle(ls)
    ls = np.array(ls)
    splited_ls = np.split(ls, fold_k)
    
    for i, l in enumerate(splited_ls):
        fold_dict[i] += list(l)
    
    
training_set = []

for i in range(fold_k):
    training_set += fold_dict[i]

In [8]:
CFG = {
    'AUG_MODE' : 'albu', # or [albu,transform]
    'MEAN' : [0.5051, 0.4853, 0.4409],
    'STD' : [0.2774, 0.2568, 0.2795],
    'CutMix': True,
    'mix_prob' : 0.5
}

## Class Weight

In [9]:
class_weight = []

dist = getDistributionDataSet(training_set)
dist = dict(sorted(dist.items(), key=lambda x:int(x[0])))

sum_of_dist = 0

for k in dist.keys():
    sum_of_dist += dist[k]

for k in dist.keys():
    class_weight.append(1 - (dist[k] / sum_of_dist))

# Transform

In [11]:
def getTransform(train_mean = [.5, .5, .5], train_std= [.5, .5, .5], val_mean= [.5, .5, .5], val_std= [.5, .5, .5], aug_mode='albu'):
    if aug_mode == 'albu':
        transform_train = A.Compose([
            A.Resize(32,32),
            A.Rotate(limit=(-360,360), interpolation=1, border_mode=1),
            A.VerticalFlip(p=0.5),
            A.HorizontalFlip(p=0.5),
            A.Normalize(train_mean, train_std),
            ToTensorV2(),
        ])
    
        transform_test = A.Compose([
            A.Resize(32,32),
            A.Normalize(val_mean, val_std),
            ToTensorV2(),
        ])
    else:
        transform_train = transforms.Compose([
            transforms.Resize(32),
            AutoAugment(AutoAugmentPolicy.CIFAR10),
            transforms.Normalize(train_mean, train_std),
            transforms.ToTensor(),
        ])
        
        transform_test = transforms.Compose([
            transforms.Resize(32),
            transforms.Normalize(val_mean, val_std),
            transforms.ToTensor(),
        ])
        
    return transform_train, transform_test

## Kfold

In [12]:
train_norm, train_std = getNormStd(training_set)
transform_train, transform_valid = getTransform(train_norm, train_std, train_norm, train_std)

In [13]:
all_labels = []
for file in training_set:
    all_labels.append(int(file.split('/')[3]))
len(all_labels), len(training_set)

(8250, 8250)

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=False)

for foldk, (train_idx, val_idx) in enumerate(kf.split(X=training_set, y=all_labels)):
    print(f'============= Fold-{foldk} strat =============')
    model = timm.create_model(model_name='xception41p', pretrained=False, num_classes=30, drop_rate=0.1)
    
    lr = 0.000
    
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(val_idx)
    
    train_dataset = CustomLoader(training_set, transforms=transform_train, is_train=True, aug_mode=CFG['AUG_MODE'])
    valid_dataset = CustomLoader(training_set, transforms=transform_valid, is_train=True, aug_mode=CFG['AUG_MODE'])
    
    train_loader = DataLoader(train_dataset, batch_size = 64, num_workers=4, sampler=train_subsampler)
    valid_loader = DataLoader(valid_dataset, batch_size = 64, num_workers=4, sampler=valid_subsampler)
    
    criterion = nn.CrossEntropyLoss(weight = torch.tensor(class_weight).to(device))
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9,0.999), weight_decay=0.03)
    scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0 = 100, T_mult=1, eta_max=0.001, T_up=10, gamma=0.5)
    
    # Train
    n_epochs = 200
    EPOCH_FROM = 0
    
    train_loss = torch.zeros(n_epochs)
    valid_loss = torch.zeros(n_epochs)
    
    train_acc = torch.zeros(n_epochs)
    valid_acc = torch.zeros(n_epochs)
    
    valid_loss_min = np.Inf
    past_lr = lr
    model.to(device)
    
    last_loss_update = 0
    
    for e in range(n_epochs):
        # Trian
        model.train()
        for (image, labels, _) in tqdm(train_loader):
            image, labels = image.to(device), labels.to(device)
            
            optimizer.zero_grad()
            
            if CFG['CutMix'] and e < n_epochs - 10:
                mix_decision = np.random.rand()
                if mix_decision  < CFG['mix_prob']:
                    image, labels = cutmix(image, labels, 1.0)
            else:
                mix_decision = 1
                
            pred = model(image)
            
            if mix_decision < CFG['mix_prob']:
                loss = criterion(pred, labels[0]) * labels[2] + criterion(pred, labels[1]) * (1-labels[2])
            else:
                loss = criterion(pred, labels)
            
            loss.backward()
            optimizer.step()
            train_loss[e] += loss.item()
    
            ps = F.softmax(pred, dim=1)
            top_p, top_class = ps.topk(1, dim=1)
            
        
        train_loss[e] /= len(train_loader)
        train_acc[e] /= len(train_loader)
        
        # Validation
        with torch.no_grad():
            model.eval()
            for image, labels, _ in tqdm(valid_loader):
                image, labels = image.to(device), labels.to(device)
                
                logits = model(image)
                loss = criterion(logits, labels)
                valid_loss[e] += loss.item()
    
                ps = F.softmax(logits, dim=1)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == labels.reshape(top_class.shape)
                valid_acc[e] += torch.mean(equals.type(torch.float)).detach().cpu()
    
        valid_loss[e] /= len(valid_loader)
        valid_acc[e] /= len(valid_loader)
        
        scheduler.step(epoch=e)
        
        print('Fold: {} \tEpoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(foldk, e, train_loss[e], valid_loss[e]))
        print('Fold: {} \tEpoch: {} \tTraining accuracy: {:.6f} \tValidation accuracy: {:.6f}'.format(foldk, e, train_acc[e], valid_acc[e]))
        
        print(optimizer.param_groups[-1]['lr'])
            
        if valid_loss[e] <= valid_loss_min:
            print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss[e]))
            torch.save(model.state_dict(), f'./models/replay_test/best_model_fold{foldk}.pth')
            valid_loss_min = valid_loss[e]
            last_loss_update = e